In [10]:
import os
import copy
import json
import re
import json
import pickle as pkl
from importlib import reload
from zipfile import ZipFile
from io import BytesIO
from collections import defaultdict

import pandas as pd
import numpy as np

from owlready2 import *
import rdflib

In [2]:
pc = get_ontology("../../data/PathwayCommons12.reactome.BIOPAX.owl").load()
#pc = get_ontology("../../data/PathwayCommons12.kegg.BIOPAX.owl").load()

In [3]:
Pathway = list(filter(lambda x: x.name == 'Pathway', pc.world.classes()))[0]
Interaction = list(filter(lambda x: x.name == 'Interaction', pc.world.classes()))[0]
Conversion = list(filter(lambda x: x.name == 'Conversion', pc.world.classes()))[0]
Control = list(filter(lambda x: x.name == 'Control', pc.world.classes()))[0]
Catalysis = list(filter(lambda x: x.name == 'Catalysis', pc.world.classes()))[0]
TemplateReaction = list(filter(lambda x: x.name == 'TemplateReaction', pc.world.classes()))[0]
BiochemicalReaction = list(filter(lambda x: x.name == 'BiochemicalReaction', pc.world.classes()))[0]
BindingFeature = list(filter(lambda x: x.name == 'BindingFeature', pc.world.classes()))[0]
ModificationFeature = list(filter(lambda x: x.name == 'ModificationFeature', pc.world.classes()))[0]
Complex = list(filter(lambda x: x.name == 'Complex', pc.world.classes()))[0]

#list(pc.world.classes())

In [5]:
def convert_db_name(db):
  if db == 'UniProtKB' or db == 'uniprot knowledgebase':
    return 'uniprot'
  elif db == 'ncbi gene':
    return 'ncbi'
  elif db == 'hgnc symbol':
    return 'hgnc'
  return db.lower()


def convert_db_id(id):
  if ':' in str(id):
    return id.split(':')[-1].lower()
  return str(id).lower()


def make_db_ref(db, id):
  return '{0}:{1}'.format(convert_db_name(db), convert_db_id(id))

def load_gene_info(path):
  """Load the NCBI Gene file and returns a {gene_name: info} dictionary"""
  gene_info = pd.read_csv(path, delimiter='\t')
  gene_info_map = {}
  gene_name_map = {}
  for _, row in gene_info.iterrows():
    info = {
      'name': row['Symbol'],
#       'type': row['type_of_gene'],
#       'desc': row['Other_designations'].split('|')[0],
    }
    ids = [make_db_ref('ncbi', row['GeneID'])]
    gene_name_map[info['name']] = info
    for xref in row['dbXrefs'].split('|'):
      xref_parts = xref.split(':')
      db = xref_parts[0]
      id = xref_parts[-1]
      ids.append(make_db_ref(db, id))
    for id in ids:
      gene_info_map[id] = info
#     for s in row['Synonyms'].split('|'):
#       gene_info_map[s] = info
  return gene_info, gene_info_map, gene_name_map

gene_info, gene_info_map, gene_name_map = load_gene_info('../../data/Mus_musculus.gene_info')

In [6]:
def parse_molecule(m):
  res = {
    'type': 'molecule',
    'name': m.displayName if m.displayName is not None else m.name,
#     'name': getattr(m, 'displayName', m.name),
#     'name': m.name,
  }
  if isinstance(m, Complex):
    res['type'] = 'complex'
    res['component'] = [parse_molecule(cc) for cc in m.component]
  if hasattr(m, 'cellularLocation') and m.cellularLocation is not None:
    if len(m.cellularLocation.term) > 0:
      res['cellularLocation'] = m.cellularLocation.term[0]
      #print('cellularLocation', res['cellularLocation'])
#   if hasattr(m, 'feature') and m.feature is not None:
#     res['feature'] = []
#     if len(m.feature) > 0:
#       for f in m.feature:
#         if not isinstance(f, ModificationFeature) and not isinstance(f, BindingFeature):
#           continue
#         feature = {
#           'biopaxType': str(f.__class__).replace('biopax-level3.', ''),
#         }
#         if hasattr(f, 'modificationType') and f.modificationType is not None:
#           feature['modificationType'] = f.modificationType.term[0]
#         if hasattr(f, 'featureLocation') and f.featureLocation is not None:
#           feature['featureLocation'] = [{
#             'sequencePosition': l.sequencePosition,
#             'positionStatus': l.positionStatus,
#           } for l in f.featureLocation]
# #         print(f.get_properties())
# #         print(f.modificationType.term[0])
# #         print(f.featureLocation[0].get_properties())
# #         print(f.featureLocation[0].sequencePosition)
# #         print(f.featureLocation[0].positionStatus)
# # #         print(f.comment)
# #         print(f.featureLocation)
# #         print(f.modificationType.term[0])
#         res['feature'].append(feature)
  if hasattr(m, 'entityReference') and m.entityReference is not None:
    e = m.entityReference
    res['entityReference'] = {
      'name': getattr(e, 'displayName', e.name),
    }
    if hasattr(e, 'xref') and e.xref is not None:
      for x in e.xref:
        if x.db is None:
          continue
        conv_id = '{0}:{1}'.format(convert_db_name(x.db), convert_db_id(x.id))
#         if conv_id in all2mouse:
#           for db_ref in all2mouse[conv_id]:
#             if db_ref in gene_info_map:
#               res['entityReference']['gene'] = gene_info_map[db_ref]
        if 'gene' not in res['entityReference']:
          if res['entityReference']['name'] in gene_name_map:
            res['entityReference']['gene'] = gene_name_map[res['entityReference']['name']]
          elif res['name'] in gene_name_map:
            res['entityReference']['gene'] = gene_name_map[res['name']]
        
        # TODO: resolve small molecules
        if ('uniprot' in x.db) \
          or ( \
            (x.db == 'ensembl' or x.db == 'ncbi gene') \
            and ( \
              'xref' not in res['entityReference'] \
              or res['entityReference']['xref']['db'] != 'ncbi gene' \
            ) \
          ) \
          or ( \
#             x.db in ['chebi'] \
#             and 
            'xref' not in res['entityReference'] \
          ):
          # NCBI Gene has the top priority and the Ensemlb goes next
          res['entityReference']['xref'] = {
            'id': convert_db_id(x.id),
            'db': convert_db_name(x.db),
          }
#       res['entityReference']['xref'] = [
#         {
#           'id': x.id,
#           'db': x.db,
# #           'relationshipType': [
# #              r.term
# #             for r in getattr(x, 'relationshipType', [])
# #           ],
#         }
#         for x in e.xref
#       ]
#     if hasattr(e, 'organism') and e.organism is not None:
#       o = e.organism
#       res['entityReference']['organism'] = {
#         'name': getattr(o, 'displayName', o.name),
#       }
#       if 'Homo' not in res['entityReference']['organism']['name']:
#         print(res['entityReference']['organism']['name'])
  return res


def parse_reaction(r):
  res = {
    'type': 'reaction',
    'name': r.displayName if r.displayName is not None else r.name,
#     'name': getattr(r, 'displayName', r.name),
#     'comment': getattr(r, 'comment', None),
    'conversionDirection': getattr(r, 'conversionDirection', None),
  }
  if hasattr(r, 'left') and r.left is not None:
    res['left'] = [parse_molecule(c) for _, c in enumerate(r.left)]
  if hasattr(r, 'right') and r.right is not None:
    res['right'] = [parse_molecule(c) for _, c in enumerate(r.right)]
  return res


def parse_control(c):
  res = {
    'type': 'control',
    'name': c.displayName if c.displayName is not None else c.name,
#     'name': getattr(c, 'displayName', c.name),
#     'comment': getattr(c, 'comment', None),
    'controlType': getattr(c, 'controlType', None),
  }
  if hasattr(c, 'controller') and c.controller is not None:
    res['controller'] = [parse_molecule(cc) for cc in c.controller]
  if hasattr(c, 'cofactor') and c.cofactor is not None:
    res['cofactor'] = [parse_molecule(cc) for cc in c.cofactor]
  if hasattr(c, 'controlled') and c.controlled is not None:
    res['controlled'] = parse_reaction(c.controlled)
  return res


def parse_template_reaction(c):
  res = {
    'type': 'template_reaction',
    'name': c.displayName if c.displayName is not None else c.name,
#     'name': getattr(c, 'displayName', c.name),
#     'comment': getattr(c, 'comment', None),
    'templateDirection': getattr(c, 'templateDirection', None),
  }
  if hasattr(c, 'template') and c.template is not None:
    res['template'] = parse_molecule(c.template)
  if hasattr(c, 'product') and c.product is not None:
    res['product'] = [parse_molecule(cc) for cc in c.product]
  return res


parsed_pathway_steps = {}
def parse_pathway_step(s):
  reactions = []
  if s.name in parsed_pathway_steps:
    # Avoid infinite recursions
    return reactions
  parsed_pathway_steps[s.name] = ''
  res = {
    'type': 'pathway_step',
    'name': s.displayName if s.displayName is not None else s.name,
#     'name': getattr(c, 'displayName', c.name),
#     'comment': getattr(c, 'comment', None),
    'stepDirection': getattr(s, 'stepDirection', None),
  }
  if hasattr(s, 'stepConversion') and s.stepConversion is not None:
    res['stepConversion'] = parse_reaction(s.stepConversion)
    reactions.append(res['stepConversion'])
  if hasattr(s, 'stepProcess') and s.stepProcess is not None:
    res['stepProcess'] = [parse_control(cc) for cc in s.stepProcess]
    reactions.extend(res['stepProcess'])
  if hasattr(s, 'nextStep') and s.nextStep is not None:
    for ss in s.nextStep:
      reactions.extend(parse_pathway_step(ss))
  return reactions

def parse_pathway(p):
  res = {
    'type': 'pathway',
    'name': getattr(p, 'displayName', p.name),
#     'comment': getattr(p, 'comment', None),
  }
  if hasattr(p, 'pathwayComponent') and p.pathwayComponent is not None:
    res['pathwayComponent'] = []
    for c in p.pathwayComponent:
      if isinstance(c, Pathway):
#         res_list.append(parse_pathway(c))
        continue
      elif isinstance(c, Control):
        res['pathwayComponent'].append(parse_control(c))
      elif isinstance(c, TemplateReaction):
        res['pathwayComponent'].append(parse_template_reaction(c))
      else:
        res['pathwayComponent'].append(parse_reaction(c))
  
  if hasattr(p, 'pathwayOrder') and p.pathwayOrder is not None:
    if 'pathwayComponent' not in res:
      res['pathwayComponent'] = []
    for s in p.pathwayOrder:
      res['pathwayComponent'].extend(parse_pathway_step(s))

  return res


# print(list(pc.world.properties()))

parsed_map = {}
pathways = []

generic_pathway_template = {
  'type': 'pathway',
  'name': '<generic>',
  'pathwayComponent': [],
}

def process_reactions(reaction_class, parse_fn):
  print(len(reaction_class.instances()))
  for c in reaction_class.instances()[:]:
    if c.name in parsed_map:
      continue
    parsed_map[c.name] = ''
#     for xref in c.xref:
#       print(xref.title)
#       print(xref.get_properties())
    r = parse_fn(c)
    generic_pathway = copy.deepcopy(generic_pathway_template)
    generic_pathway['pathwayComponent'].append(r)
    pathways.append(generic_pathway) 

for reaction_class, parse_fn in [
  (Control, parse_control),
  (TemplateReaction, parse_template_reaction),
  (BiochemicalReaction, parse_reaction),
  (Conversion, parse_reaction),
#   (Interaction, parse_reaction),
]:
  process_reactions(reaction_class, parse_fn)

# for p in Pathway.instances()[:]:
#   pathway = parse_pathway(p)
# #   print(json.dumps(res, indent=2)[:100000])
#   pathways.append(pathway)

print(len(pathways))

7425
14
12184
12201
19640


In [21]:
def match_participant(m, query):
  if m['type'] == 'complex':
    return any([match_participant(c, query) for c in m['component']])
  if query.lower() in m['name'].lower():
    return True
  return False

def filter_pathways_by_participant(query):
  filtered_pathways = []
  for p in pathways:
    filtered_p = copy.deepcopy(p)
    filtered_p['pathwayComponent'] = []
    for r in p['pathwayComponent']:
      matched = False
      rr = r
      if r['type'] == 'control':
        for m in r['controller']:
          matched = matched or match_participant(m, query)
        for m in r['cofactor']:
          matched = matched or match_participant(m, query)
      elif r['type'] == 'template_reaction':
        for m in r['product']:
          matched = matched or match_participant(m, query)
      if 'left' in rr:
        for m in rr['left']:
          matched = matched or match_participant(m, query)
      if 'right' in rr:
        for m in rr['right']:
          matched = matched or match_participant(m, query)
    if matched:
      filtered_p['pathwayComponent'].append(r)
    if len(filtered_p['pathwayComponent']) > 0:
      filtered_pathways.append(filtered_p)
  return filtered_pathways

filtered_pathways = filter_pathways_by_participant('ca2+')
print(len(filtered_pathways))
print(json.dumps(list(map(lambda p: p['pathwayComponent'][0]['name'], filtered_pathways)), indent=2))
#print(json.dumps(list(map(lambda p: p['pathwayComponent'][0]['controlled']['name'], filtered_pathways)), indent=2))
#print(json.dumps(filtered_pathways[:5], indent=2))

# https://reactome.org/PathwayBrowser/#/R-HSA-163200&SEL=R-HSA-163213&PATH=R-HSA-1430728,R-HSA-1428517
# https://reactome.org/PathwayBrowser/#/R-HSA-163200&SEL=R-HSA-163214&PATH=R-HSA-1430728,R-HSA-1428517

# for p in pathways:
#   for r in p['pathwayComponent']:
#     print(r['name'])
#     break
#     if 'R11945' in r['name']:
#       print(r)

339
[
  "R-HSA-158645",
  "R-HSA-5336160",
  "R-HSA-5607785",
  "R-HSA-2648999",
  "R-HSA-2730672",
  "R-HSA-5358685",
  "R-HSA-2684957",
  "R-HSA-111972",
  "R-HSA-3229230",
  "R-HSA-5607784",
  "R-HSA-5607787",
  "R-HSA-5610119",
  "R-HSA-8857690",
  "R-HSA-5333036",
  "Control_92e7377f8c17b64a542bbdb675a3fe12",
  "R-HSA-1489501",
  "R-HSA-1663719",
  "R-HSA-9034483",
  "R-HSA-5336148",
  "R-HSA-9015052",
  "Control_0f0070d86be5ad0b29e8311d985db98c",
  "R-HSA-158643",
  "R-HSA-977428",
  "R-HSA-210503",
  "Control_0224ec86f778e7febf2593ac5ddcc3b0",
  "R-HSA-5333034",
  "R-HSA-2855230",
  "R-HSA-5211366",
  "R-HSA-9034487",
  "R-HSA-5626550",
  "R-HSA-3223249",
  "R-HSA-5607786",
  "R-HSA-9034486",
  "R-HSA-9034527",
  "R-HSA-8876920",
  "Control_656defa0440dd838b21dca1ca1196c61",
  "R-HSA-71589",
  "R-HSA-71542",
  "R-HSA-2744259",
  "R-HSA-6809098",
  "R-HSA-8848729",
  "R-HSA-3223247",
  "R-HSA-5607091",
  "R-HSA-2889054",
  "R-HSA-9015053",
  "Catalysis_33509413b4323e37244bb4583dc

In [1]:
import Bio.Data.CodonTable

In [7]:
(Bio.Data.CodonTable.standard_dna_table).forward_table

{'TTT': 'F',
 'TTC': 'F',
 'TTA': 'L',
 'TTG': 'L',
 'TCT': 'S',
 'TCC': 'S',
 'TCA': 'S',
 'TCG': 'S',
 'TAT': 'Y',
 'TAC': 'Y',
 'TGT': 'C',
 'TGC': 'C',
 'TGG': 'W',
 'CTT': 'L',
 'CTC': 'L',
 'CTA': 'L',
 'CTG': 'L',
 'CCT': 'P',
 'CCC': 'P',
 'CCA': 'P',
 'CCG': 'P',
 'CAT': 'H',
 'CAC': 'H',
 'CAA': 'Q',
 'CAG': 'Q',
 'CGT': 'R',
 'CGC': 'R',
 'CGA': 'R',
 'CGG': 'R',
 'ATT': 'I',
 'ATC': 'I',
 'ATA': 'I',
 'ATG': 'M',
 'ACT': 'T',
 'ACC': 'T',
 'ACA': 'T',
 'ACG': 'T',
 'AAT': 'N',
 'AAC': 'N',
 'AAA': 'K',
 'AAG': 'K',
 'AGT': 'S',
 'AGC': 'S',
 'AGA': 'R',
 'AGG': 'R',
 'GTT': 'V',
 'GTC': 'V',
 'GTA': 'V',
 'GTG': 'V',
 'GCT': 'A',
 'GCC': 'A',
 'GCA': 'A',
 'GCG': 'A',
 'GAT': 'D',
 'GAC': 'D',
 'GAA': 'E',
 'GAG': 'E',
 'GGT': 'G',
 'GGC': 'G',
 'GGA': 'G',
 'GGG': 'G'}

In [21]:
mol_data = []
type_id = 0

mol_data.extend([
  {
    'type': type_id + 0,
    'category': 'lipid',
    'name': 'Generic lipid',
    'radius': 0.0025,
  },
  {
    'type': type_id + 1,
    'category': 'protein',
    'name': 'Generic protein',
    'radius': 0.0025,
  }
])

type_id = 100
# Generic RNA codons
mol_data.extend([
  {
    'type': type_id + i,
    'category': 'rna',
    'name': codon,
    'radius': 0.00075,
  }
  for i, (codon, amino_acid) in enumerate(Bio.Data.CodonTable.standard_dna_table.forward_table.items())
])
type_id += len(Bio.Data.CodonTable.standard_dna_table.forward_table)

# Start/stop RNA codons
mol_data.extend([
  {
    'type': type_id + i,
    'category': 'rna',
    'name': codon,
    'radius': 0.00075,
  }
  for i, codon in enumerate(Bio.Data.CodonTable.standard_dna_table.start_codons + Bio.Data.CodonTable.standard_dna_table.stop_codons)
])

# Write to a file
with open('./gen/large-molecules.json', 'wt') as f:
  f.write(json.dumps(mol_data, indent=2))